In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24830
2,Huelva,Confirmados PDIA 14 días,269
3,Huelva,Tasa PDIA 14 días,"52,419276263226614"
4,Huelva,Confirmados PDIA 7 días,141
5,Huelva,Total Confirmados,25035
6,Huelva,Curados,18115
7,Huelva,Fallecidos,343


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24830.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6596.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 5753 personas en los últimos 7 días 

Un positivo PCR cada 3060 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24830.0,141.0,269.0,513170.0,27.476275,52.419276,34.0
Condado-Campiña,6980.0,63.0,108.0,156231.0,40.324904,69.128406,17.0
Huelva-Costa,15136.0,74.0,155.0,289548.0,25.557075,53.531711,16.0
Huelva (capital),6596.0,25.0,47.0,143837.0,17.380785,32.675876,8.0
Ayamonte,1227.0,16.0,36.0,21104.0,75.815011,170.583776,4.0
Gibraleón,515.0,4.0,4.0,12737.0,31.404569,31.404569,4.0
Villalba del Alcor,451.0,6.0,6.0,3366.0,178.253119,178.253119,4.0
Cartaya,963.0,7.0,20.0,20083.0,34.855350,99.586715,3.0
Almonte,880.0,8.0,22.0,24507.0,32.643734,89.770270,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,102.0,7.0,17.0,3092.0,226.390686,549.805951,0.0
San Bartolomé de la Torre,206.0,7.0,13.0,3761.0,186.120713,345.652752,2.0
Lucena del Puerto,162.0,5.0,11.0,3261.0,153.327200,337.319841,0.0
San Silvestre de Guzmán,13.0,2.0,2.0,614.0,325.732899,325.732899,0.0
Corteconcepción,24.0,1.0,1.0,536.0,186.567164,186.567164,0.0
Villalba del Alcor,451.0,6.0,6.0,3366.0,178.253119,178.253119,4.0
Ayamonte,1227.0,16.0,36.0,21104.0,75.815011,170.583776,4.0
Palma del Condado (La),985.0,7.0,11.0,10801.0,64.808814,101.842422,2.0
Cartaya,963.0,7.0,20.0,20083.0,34.855350,99.586715,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,123.0,1.0,3.0,3933.0,25.425884,76.277651,0.0,0.333333
Rociana del Condado,313.0,1.0,3.0,7939.0,12.596045,37.788135,0.0,0.333333
Cartaya,963.0,7.0,20.0,20083.0,34.855350,99.586715,3.0,0.350000
Almonte,880.0,8.0,22.0,24507.0,32.643734,89.770270,2.0,0.363636
Bollullos Par del Condado,621.0,4.0,10.0,14387.0,27.802878,69.507194,0.0,0.400000
Puebla de Guzmán,102.0,7.0,17.0,3092.0,226.390686,549.805951,0.0,0.411765
Ayamonte,1227.0,16.0,36.0,21104.0,75.815011,170.583776,4.0,0.444444
Lucena del Puerto,162.0,5.0,11.0,3261.0,153.327200,337.319841,0.0,0.454545
Huelva-Costa,15136.0,74.0,155.0,289548.0,25.557075,53.531711,16.0,0.477419
